## 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 

'''
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 3114: invalid continuation byte
에러를 막기 위해 뒤에 인코딩될 언어를 지정해줌
'''
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2. 분석해보기
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [6]:
ratings['movie_id'].nunique()

3628

In [7]:
ratings['user_id'].nunique()

6039

참고 : pandas DataFrame에서 특정 행, 열만 출력하기
https://blog.naver.com/PostView.nhn?blogId=rising_n_falling&logNo=221622971970&proxyReferer=https:%2F%2Fwww.google.com%2F

pandas merge를 통해 두개의 데이터 합치기
https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

pandas groupby sum 
https://stackoverflow.com/questions/39922986/pandas-group-by-and-sum

In [8]:
# print(movies.loc[0][1])
merged_df = pd.merge(ratings, movies, on = "movie_id")

movie_count = merged_df.groupby('title')['user_id'].count()

movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [9]:
merged_df

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [10]:
# 인기 많은 아티스트
artist_count = ratings.groupby('movie_id')['user_id'].count()
artist_count.sort_values(ascending=False).head(150)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
        ... 
2100    1030
364     1029
1234    1028
2529    1027
1258    1024
Name: user_id, Length: 150, dtype: int64

In [11]:
#idx_to_movie = {i:v for i,v in zip(movies['movie_id'], movies['title'])}
#movie_to_idx = {v:i for i,v in zip(movies['movie_id'], movies['title'])}

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [12]:
my_favorite = ['Men in Black (1997)', 'Aladdin (1992)','Mission: Impossible (1996)','Lion King, The (1994)', 'Terminator 2: Judgment Day (1991)' ]

In [13]:
merged_df['user_id'].nunique()

6039

In [14]:
my_ratings = pd.DataFrame({'user_id': ['bk']*5, 'title': my_favorite, 'count':[5]*5})

if not merged_df.isin({'user_id':['bk']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    merged_df = merged_df.append(my_ratings)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

merged_df.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,bk,NaN,5,NaN,Men in Black (1997),NaN
1,bk,NaN,5,NaN,Aladdin (1992),NaN
2,bk,NaN,5,NaN,Mission: Impossible (1996),NaN
3,bk,NaN,5,NaN,"Lion King, The (1994)",NaN
4,bk,NaN,5,NaN,Terminator 2: Judgment Day (1991),NaN


In [15]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = merged_df['user_id'].unique()
title_unique = merged_df['title'].unique()
movie_id_unique = merged_df['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}
movie_id_to_idx = {v:k for k,v in enumerate(movie_id_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['bk'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
#print(title_to_idx['Men in Black'])

6039


In [16]:
temp_user_data = merged_df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(merged_df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    merged_df['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = merged_df['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(merged_df):
    print('movie_id column indexing OK!!')
    merged_df['movie_id'] = temp_title_data
else:
    print('movie_id column indexing Fail!!')

merged_df

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count,timestamp,title,genre
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,0,5,978298413.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,2,0,4,978220179.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,3,0,4,978199279.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,4,0,5,978158471.0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
0,6039,175,5,NaN,Men in Black (1997),NaN
1,6039,33,5,NaN,Aladdin (1992),NaN
2,6039,58,5,NaN,Mission: Impossible (1996),NaN
3,6039,330,5,NaN,"Lion King, The (1994)",NaN


## 4. CSR matrix를 만들기

In [17]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = merged_df['user_id'].nunique()
num_movie = merged_df['movie_id'].nunique()

csr_data = csr_matrix((merged_df['count'], (merged_df.user_id, merged_df.movie_id)))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델을 구성하여 훈련

In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [19]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [20]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [21]:
als_model.fit(csr_data_transpose)

## 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [22]:
bk, Lion_king = user_to_idx['bk'], title_to_idx['Lion King, The (1994)']
bk_vector, Lion_king_vector = als_model.user_factors[bk], als_model.item_factors[Lion_king]

print('슝=3')

슝=3


In [23]:
bk_vector

array([ 0.510733  ,  0.4149558 ,  0.26287472, -0.7471573 , -0.18509237,
       -0.45504624, -0.08459511, -0.23128477,  0.7195565 , -0.96013665,
        0.8781102 ,  0.3368516 ,  0.34261826, -0.8153166 , -0.6332142 ,
        0.409747  , -0.0356956 , -0.5443367 ,  0.20156156, -0.39464155,
        0.7795721 , -0.2889115 , -0.9721602 ,  0.6126084 ,  0.38607755,
       -0.82931405, -0.4158374 , -0.07626424, -0.05685828,  0.9555321 ,
        0.62541753,  0.946113  , -0.04747085,  0.32337546,  0.25639448,
       -0.71600276, -0.01021122, -0.46155754, -0.53963697,  0.12579893,
        0.14698765,  1.1001325 , -0.10925755,  0.6697179 , -0.93961406,
       -0.39879057, -0.71837604,  0.3275054 ,  0.7230766 , -0.27632824,
        1.0582138 , -0.81586933, -0.8419613 ,  1.0072238 ,  0.0340302 ,
       -0.00425453, -0.8011754 , -0.06131786,  0.82460326,  0.71648705,
       -0.28285757,  0.05496889,  0.33102992,  0.26291624, -0.12887332,
       -0.10019515,  0.36498564, -0.32337955, -0.4724111 ,  0.40

In [24]:
Lion_king_vector

array([ 0.00578281, -0.00770213,  0.00424199, -0.00568464,  0.00066872,
       -0.00412301,  0.00594896,  0.01786908,  0.0209209 , -0.00193029,
        0.0268314 ,  0.02160212,  0.01050164,  0.00765528, -0.02507179,
        0.00258002, -0.00977509,  0.00624424,  0.00799564,  0.00097455,
        0.00010118,  0.00733268, -0.01094606,  0.0095056 ,  0.01578609,
       -0.0168321 , -0.01122902,  0.01245564,  0.00908844,  0.02424104,
        0.02521921,  0.03016615,  0.03635892, -0.00557844,  0.01011587,
        0.00135126,  0.00013614, -0.03116798, -0.01771405, -0.01547297,
        0.03046264,  0.03096852, -0.00567592,  0.00863606, -0.00411036,
        0.02322158,  0.00373038,  0.03884446,  0.01786585, -0.00176283,
        0.03696612, -0.02652097,  0.00391443,  0.03583474,  0.01646124,
       -0.01292253,  0.00798135,  0.01898304,  0.01015375,  0.01302174,
        0.00040361,  0.03715385,  0.01691104, -0.00215891,  0.00123815,
        0.01598348,  0.0261718 , -0.02990604, -0.0067194 ,  0.00

In [25]:
# bk와 Lion king를 내적하는 코드
np.dot(bk_vector, Lion_king_vector)

0.5405338

In [26]:
queen = title_to_idx['Aladdin (1992)']
queen_vector = als_model.item_factors[queen]
np.dot(bk_vector, queen_vector)

0.5456425

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천

In [28]:
favorite_title = 'Lion King, The (1994)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(330, 0.9999999),
 (10, 0.89502895),
 (33, 0.784847),
 (34, 0.6612912),
 (35, 0.6549535),
 (8, 0.6039119),
 (329, 0.60344374),
 (191, 0.59488416),
 (16, 0.5841153),
 (32, 0.58338463),
 (30, 0.56269795),
 (25, 0.5527592),
 (37, 0.53326756),
 (46, 0.518192),
 (548, 0.5138861)]

In [29]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Aladdin (1992)',
 'Mulan (1998)',
 'Hunchback of Notre Dame, The (1996)',
 'Snow White and the Seven Dwarfs (1937)',
 'Anastasia (1997)',
 'Little Mermaid, The (1989)',
 'Tarzan (1999)',
 'Hercules (1997)',
 'Antz (1998)',
 'Pocahontas (1995)',
 'Cinderella (1950)',
 'Dumbo (1941)',
 'Fantasia (1940)']

위 내용을 함수화 함

In [30]:
def get_similar_movie(title: str):
    title_id = title_to_idx[title]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print("슝=3")

슝=3


In [31]:
get_similar_movie('Lion King, The (1994)')

['Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Aladdin (1992)',
 'Mulan (1998)',
 'Hunchback of Notre Dame, The (1996)',
 'Snow White and the Seven Dwarfs (1937)',
 'Anastasia (1997)',
 'Little Mermaid, The (1989)',
 'Tarzan (1999)',
 'Hercules (1997)']

## 8. 내가 가장 좋아할 만한 영화를 추천받기

In [32]:
user = user_to_idx['bk']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(107, 0.63345915),
 (10, 0.5259025),
 (62, 0.42719537),
 (124, 0.4160337),
 (40, 0.32627788),
 (191, 0.29188707),
 (284, 0.27741843),
 (138, 0.2710719),
 (141, 0.26217616),
 (60, 0.2546242),
 (82, 0.23723705),
 (145, 0.23489076),
 (1, 0.22822857),
 (34, 0.22466016),
 (87, 0.22099508),
 (236, 0.21210708),
 (179, 0.21117786),
 (32, 0.2009778),
 (841, 0.20010394),
 (150, 0.1967935)]

In [33]:
[idx_to_title[i[0]] for i in artist_recommended]

['Jurassic Park (1993)',
 'Beauty and the Beast (1991)',
 'Total Recall (1990)',
 'Matrix, The (1999)',
 'Toy Story (1995)',
 'Little Mermaid, The (1989)',
 'Nightmare Before Christmas, The (1993)',
 'True Lies (1994)',
 'Fugitive, The (1993)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Lost World: Jurassic Park, The (1997)',
 'Fifth Element, The (1997)',
 'James and the Giant Peach (1996)',
 'Mulan (1998)',
 'Braveheart (1995)',
 'Speed (1994)',
 'Rock, The (1996)',
 'Hercules (1997)',
 'Prince of Egypt, The (1998)',
 'Independence Day (ID4) (1996)']

In [36]:
Jurassic = title_to_idx['Jurassic Park (1993)']
explain = als_model.explain(user, csr_data, itemid=Jurassic)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Men in Black (1997)', 0.30144285084874545),
 ('Terminator 2: Judgment Day (1991)', 0.21346755093180364),
 ('Mission: Impossible (1996)', 0.10520431719110221),
 ('Lion King, The (1994)', 0.01869950226228621),
 ('Aladdin (1992)', -0.014783003358950159)]